In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import random
import requests
import eventlet

In [2]:
# Open the wensite and log in
options = webdriver.ChromeOptions()
## Change the executable_path to where you installed your Chromedriver
driver = webdriver.Chrome(executable_path='C:/Program Files (x86)/Google/Chrome/Application/chromedriver', options=options)
driver.get('https://www.glassdoor.com/member/home/companies.htm')
time.sleep(1)
search_input1 = driver.find_element_by_id('userEmail')
search_input1.click()
# Change it to your email
search_input1.send_keys('')  
time.sleep(1)        
search_input2 = driver.find_element_by_id('userPassword')
search_input2.click()  
# Change it to your password
search_input2.send_keys('')
driver.find_element_by_name('submit').click()

In [86]:
# Search a company
try:
    company = driver.find_element_by_xpath('//*[@id="sc.keyword"]')
except:
    company = driver.find_element_by_xpath('//*[@id="KeywordSearch"]')
company.click()
company.send_keys('Invesco Ltd')
company.send_keys(Keys.ENTER)
time.sleep(random.random())

In [6]:
# The crawl function for the first kind of page
def get_jobs1(num_jobs):
    
    #driver.find_element_by_xpath('//*[@id="CovidInfoBanner"]/div/div/span/svg').click()
    
    jobs = []
    while len(jobs) < num_jobs:
        job_buttons = driver.find_elements_by_class_name("JobDetailsStyles__jobTitle")
        for job_button in job_buttons:
            print("Progress: {}".format("" + str(len(jobs)+1) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            try:
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass

            job_button.click()
            windows = driver.window_handles
            driver.switch_to.window(windows[-1])
            collected_successfully = False

            try:
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass


            eventlet.monkey_patch()
            with eventlet.Timeout(60,False):
                while not collected_successfully:
                    try:
                        company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                        location = driver.find_element_by_xpath('.//div[@class="location"]').text
                        job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                        job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                        collected_successfully = True
                    except:
                        time.sleep(5*random.random())

                try:
                    salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
                except NoSuchElementException:
                    salary_estimate = -1

                try:
                    rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
                except NoSuchElementException:
                    rating = -1

                #Printing for debugging
                if False:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Job Description: {}".format(job_description[:500]))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))

                #Going to the Company tab...
                #clicking on this:
                #<div class="tab" data-tab-type="overview"><span>Company</span></div>
                try:
                    driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                    try:
                        headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                    except NoSuchElementException:
                        headquarters = -1

                    try:
                        size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                    except NoSuchElementException:
                        size = -1

                    try:
                        founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                    except NoSuchElementException:
                        founded = -1

                    try:
                        type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                    except NoSuchElementException:
                        type_of_ownership = -1

                    try:
                        industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                    except NoSuchElementException:
                        industry = -1

                    try:
                        sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                    except NoSuchElementException:
                        revenue = -1

                    try:
                        competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                    except NoSuchElementException:
                        competitors = -1

                except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    headquarters = -1
                    size = -1
                    founded = -1                    
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1
                    competitors = -1


                if False:                    
                    print("Headquarters: {}".format(headquarters))
                    print("Size: {}".format(size))                
                    print("Founded: {}".format(founded))
                    print("Type of Ownership: {}".format(type_of_ownership))
                    print("Industry: {}".format(industry))
                    print("Sector: {}".format(sector))
                    print("Revenue: {}".format(revenue))
                    print("Competitors: {}".format(competitors))
                    print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

                jobs.append({"Job Title" : job_title,
                             "Salary Estimate" : salary_estimate,
                             "Job Description" : job_description,
                             "Rating" : rating,
                             "Company Name" : company_name,
                             "Location" : location,
                             "Headquarters" : headquarters,
                             "Size" : size,
                             "Founded" : founded,
                             "Type of ownership" : type_of_ownership,
                             "Industry" : industry,
                             "Sector" : sector,
                             "Revenue" : revenue,
                             "Competitors" : competitors})

            #add job to jobs
            time.sleep(5*random.random())
            windows1 = driver.window_handles
            if len(windows1) == 1:
                driver.switch_to.window(windows[0])
            else:
                driver.close()
                driver.switch_to.window(windows[0])

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('//*[@id="JobsLandingPage"]/div/div/div/div/div[5]/div/ul/li[last()]/a').click()
            driver.refresh()
        except (NoSuchElementException,ElementClickInterceptedException) as e:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    return pd.DataFrame(jobs)

In [3]:
# The crawl function for the second kind of page
def get_jobs2(num_jobs):
    
    jobs = []
    while len(jobs) < num_jobs:
        job_buttons = driver.find_elements_by_class_name("JobDetailsStyles__jobTitle")
        for job_button in job_buttons:
            print("Progress: {}".format("" + str(len(jobs)+1) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            try:
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass

            job_button.click()
            windows = driver.window_handles
            driver.switch_to.window(windows[-1])
            collected_successfully = False
            try:
                driver.find_element_by_xpath('//*[@id="JAModal"]/div/div[2]/span').click()
            except NoSuchElementException:
                pass

            eventlet.monkey_patch()
            with eventlet.Timeout(60,False):
                while not collected_successfully:
                    try:
                        company_name = driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[1]').text
                        location = driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[3]').text
                        job_title = driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[2]').text
                        job_description = driver.find_element_by_xpath('//*[@id="JobDescriptionContainer"]').text
                        collected_successfully = True
                    except:
                        time.sleep(random.random())

                try:
                    salary_estimate = driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[4]/span"]').text
                except NoSuchElementException:
                    salary_estimate = -1

                try:
                    rating = driver.find_element_by_xpath('//*[@id="JobView"]/div[1]/div[2]/div/div/div[2]/div[1]/div[1]/div[2]/div/div/div[1]/span').text
                except NoSuchElementException:
                    rating = -1

                #Printing for debugging
                if False:
                    print("Job Title: {}".format(job_title))
                    print("Salary Estimate: {}".format(salary_estimate))
                    print("Job Description: {}".format(job_description[:500]))
                    print("Rating: {}".format(rating))
                    print("Company Name: {}".format(company_name))
                    print("Location: {}".format(location))

                #Going to the Company tab...
                try:
                    try:
                        driver.find_element_by_xpath('//*[@id="JobView"]/div[3]/header/div/div/div[2]/span').click()
                    except:
                        driver.find_element_by_xpath('//*[@id="JobView"]/div[4]/header/div/div/div[2]/span').click()

                    try:
                        headquarters = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[1]/span').text
                    except NoSuchElementException:
                        headquarters = -1

                    try:
                        size = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[2]/span').text
                    except NoSuchElementException:
                        size = -1

                    try:
                        founded = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[3]/span').text
                    except NoSuchElementException:
                        founded = -1

                    try:
                        type_of_ownership = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[4]/span').text
                    except NoSuchElementException:
                        type_of_ownership = -1

                    try:
                        industry = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[5]/span').text
                    except NoSuchElementException:
                        industry = -1

                    try:
                        sector = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[6]/span').text
                    except NoSuchElementException:
                        sector = -1

                    try:
                        revenue = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[7]/span').text
                    except NoSuchElementException:
                        revenue = -1

                    try:
                        competitors = driver.find_element_by_xpath('//*[@id="InfoFields"]/div[8]/span').text
                    except NoSuchElementException:
                        competitors = -1


                except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                    headquarters = -1
                    size = -1
                    founded = -1                    
                    type_of_ownership = -1
                    industry = -1
                    sector = -1
                    revenue = -1
                    competitors = -1

                jobs.append({"Job Title" : job_title,
                             "Salary Estimate" : salary_estimate,
                             "Job Description" : job_description,
                             "Rating" : rating,
                             "Company Name" : company_name,
                             "Location" : location,
                             "Headquarters" : headquarters,
                             "Size" : size,
                             "Founded" : founded,
                             "Type of ownership" : type_of_ownership,                             
                             "Industry" : industry,
                             "Sector" : sector,
                             "Revenue" : revenue,
                             "Competitors" : competitors})

                #add job to jobs
            time.sleep(5*random.random())
            windows1 = driver.window_handles
            if len(windows1) == 1:
                driver.switch_to.window(windows[0])
            else:
                driver.close()
                driver.switch_to.window(windows[0])
        
        try:
            driver.find_element_by_xpath('//*[@id="JobsLandingPage"]/div/div/div/div/div[5]/div/ul/li[last()]/a').click()
            driver.refresh()
        except (NoSuchElementException,ElementClickInterceptedException) as e:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break
    return pd.DataFrame(jobs)

In [106]:
f = open("companies.txt","r")

companies = []
for line in f:
    company = line.split()
    company = company[0:len(company)-1]
    company = ' '.join(company)
    companies.append(company)
    
f.close()

In [117]:
errorCompanies = []
for c in companies:
    print(c)
    driver.execute_script("window.scrollBy(0,-10000)")
    time.sleep(3*random.random())
    company = driver.find_element_by_xpath('//*[@id="sc.keyword"]')
    company.click()
    company.clear()
    company.send_keys(c)
    company.send_keys(Keys.ENTER)
    
    try:
        time.sleep(3*random.random())
        driver.find_element_by_xpath('//*[@id="EIProductHeaders"]/div/a[2]').click()
        number = 500
        
        time.sleep(3*random.random())
        driver.find_elements_by_class_name("JobDetailsStyles__jobTitle")[0].click()
        windows = driver.window_handles
        driver.switch_to.window(windows[-1])
        time.sleep(3*random.random())
        new_jobs = driver.find_elements_by_class_name("jl")
        driver.close()
        driver.switch_to.window(windows[0])
        
        if len(new_jobs) > 1:
            df = get_jobs1(number)
        else:
            df = get_jobs2(number)
        df.to_csv(str(c)+'.csv', index = False)
        
    except:
        errorCompanies.append(c)

Lennar Corp
Progress: 1/500
Progress: 2/500
Progress: 3/500
Progress: 4/500
Progress: 5/500
Progress: 6/500
Progress: 7/500
Progress: 8/500
Progress: 9/500
Progress: 10/500
Progress: 11/500
Progress: 12/500
Progress: 13/500
Progress: 14/500
Progress: 15/500
Progress: 16/500
Progress: 17/500
Progress: 18/500
Progress: 19/500
Progress: 20/500
Progress: 21/500
Progress: 22/500
Progress: 23/500
Progress: 24/500
Progress: 25/500
Progress: 26/500
Progress: 27/500
Progress: 28/500
Progress: 29/500
Progress: 30/500
Progress: 31/500
Progress: 32/500
Progress: 33/500
Progress: 34/500
Progress: 35/500
Progress: 36/500
Progress: 37/500
Progress: 38/500
Progress: 39/500
Progress: 40/500
Progress: 41/500
Progress: 42/500
Progress: 43/500
Progress: 44/500
Scraping terminated before reaching target number of jobs. Needed 500, got 44.
LKQ Corp
Progress: 1/500
Progress: 2/500
Progress: 3/500
Progress: 4/500
Progress: 5/500
Progress: 6/500
Progress: 7/500
Progress: 8/500
Progress: 9/500
Progress: 10/500


In [111]:
# Then do the find out what caused the error for error company